# Overview

This notebook outlines a workflow and executes key code for batch processing of autotracking.



# Project paths
Specify the root and code paths for the project

In [7]:
import os
import platform 

"""
Defines the root path for the project, as well as the path to the code directory that holds kineKit. 
root_path is the path to the folder containing the data, and video folders.
"""

# Matt's laptop
if (platform.system() == 'Darwin') and (os.path.expanduser('~')=='/Users/mmchenry'):
    
    code_path = '/Users/mmchenry/Documents/code'
    root_path = '/Users/mmchenry/Documents/Projects/waketracking'

# Matt on Linux
elif (platform.system() == 'Linux') and (os.path.expanduser('~')=='/home/mmchenry'):

    code_path = '/home/mmchenry/code'
    root_path = '/home/mmchenry/Documents/wake_tracking'

# Ashley on Linux
elif (platform.system() == 'Linux') and (os.path.expanduser('~')=='/home/anpetey'):

    code_path = '/home/anpetey/Documents/Repositories'
    root_path = '/vortex/schooling/TRex'


# Catch alternatives
else:
    raise ValueError('Do not recognize this account -- add lines of code to define paths here')


# Packages and parameters
You'll need to execute the next cell for all of the code that follows. Make sure that the root_code and root_proj paths are correct for your system.

In [19]:
import sys
import os
import def_acquisition as da
import def_paths as dp
import numpy as np
import pandas as pd
import cv2 as cv

# The project name need to match a directory name within the root path
# proj_name = 'scaleBNT'
proj_name = 'scaleRN'

# Date of videos to analyze
# vid_date = '02-14-2023'

# Get paths (specific to system running code)
path = dp.give_paths(root_path, proj_name, code_path)

# Add path to kineKit 'sources' directory
sys.path.insert(0, path['kinekit'] + os.sep + 'sources')

# Import from kineKit
import acqfunctions as af
import videotools as vt

# Raw video extension
vid_ext_raw = 'MOV'
# vid_ext_raw = 'mp4'

# Compressed video quality level (0 to 1, where 1 is no compression)
vid_quality = 0.75

# Create mask

You'll need to create a new mask whenever the position of the tank changes in the video frame. Follow the following steps to create a new mask.

## Measure the region-of-interest (ROI)

Here the aim is to define an ellipical region-of-interest where the tank resides within the video frames.

1. Set the 'make_video' column in the 'experiment_log.csv' file to a value of 1 for the video for which you want to create a mask.

1. Set all other 'make_video' rows to 0. If there is more than one row set to 1 then the code will use the first in the list.

1. Run the cell below to save a single frame from the video, which is saved to the 'masks' directory.

1. After the frame is generated, open it up in [Fiji](https://imagej.net/software/fiji/downloads) (or ImageJ), use the ellipse tool to draw around the tank margin. Be sure not to cut off any portion of the tank where a fish might end up. All areas outside of the ROI will be ignored.

1. Once the ellipse has been drawn, select Analyze:Measure in the pull-down menus to find the region-of-interest coordinates.

1. Enter the value for 'BX' as 'roi_x' in the experiment_log (enter values online in Google Sheets). Do the same for BY->roi_y, Width->roi_w, Height->roi_h. Copy and paste the values to all rows corresponding to videos that use that same ROI.

1. Make sure that the local version of experiment_log matches the values as the Google Sheet. This can be done by redownloading the CSV file, or copy and pasting values to the local copy. 



In [18]:
# Index of video in cat list to extract video
vid_index = 0

# Extract experiment catalog info
cat = af.get_cat_info(path['cat'])
if len(cat) == 0:
    raise ValueError('No videos found in catalog file. Analyze make be set to zero for all')

# Filename for frame of current sequence
filename = cat.date[vid_index] + '_sch' + format(int(cat.sch_num[vid_index]),'02') + '_tr' + format(int(cat.trial_num[vid_index]),'03') + '_frame1'


# Define path for video
full_path = path['vidin']+ os.sep + cat.date[vid_index] + os.sep + cat.video_filename[vid_index] + '.' + vid_ext_raw

# Path for video frame
frame_path = path['mask'] + os.sep + filename + '.jpg'

# Extract frame and save to 'mask' directory
im = vt.get_frame(full_path)
result = cv.imwrite(frame_path, im)

if result is False:
    print('Save to the following path failed: ' + frame_path)
else:
    print('Video frame saved to: ' + frame_path)

Video frame saved to: /vortex/schooling/TRex/scaleRN/masks/2023-04-22_sch03_tr001_frame1.jpg


## Create the mask image

1. As in step above, the cell below will use any row for which make_video=1 in the 'experiment_log.csv' file to define the ROI for the mask, so adjust the spreadsheet accordingly.

1. The code will prompt you to choose a filename for the mask image and will save that file to the 'masks' directory.

1. Once completed, enter the mask filename (without the 'png' extension) into the mask_filename column of experiment_log for all experiments that should use that mask.

In [10]:
# Extract experiment catalog info
cat = af.get_cat_info(path['cat'])
if len(cat) == 0:
    raise ValueError('No videos found in catalog file. Analyze make be set to zero for all')

# For loop that goes through each video in the catalog
for vid_index in range(len(cat)):

    full_path = path['vidin'] + os.sep + cat.date[vid_index] + os.sep + cat.video_filename[vid_index] + '.' + vid_ext_raw

    # # Extract video frame 
    im = vt.get_frame(full_path)

    # Define roi coordinates
    roi = np.zeros(4)
    roi[0] = float(cat.roi_x[vid_index])
    roi[1] = float(cat.roi_y[vid_index])
    roi[2] = float(cat.roi_w[vid_index])
    roi[3] = float(cat.roi_h[vid_index])

    # Save mask
    da.make_mask(im, roi, path['mask'], cat.date[vid_index], cat.sch_num[vid_index], int(cat.trial_num[vid_index]))



Mask image saved to: /vortex/schooling/TRex/scaleRN/masks/2023-04-22_sch002_tr002_mask.png


# Create compressed movies

The code below will generate compressed videos for all experiments in experiment_log where analyze=1 and make_video=1. 
This is done in three steps.
First, uncompressed masked movies are created and stored in the 'tmp' directory, then compressed and cropped movies are saved in 'pilot_compressed' (the tmp movies are then deleted), and finally there is cleanup step, where the final videos are verified and tmp videos are deleted.

In [10]:
# The default is to not parallelize the code
para_mode = False

## Converting in batches

To speed up the conversion for batches of videos, we've parallelized the code (though you can skip this complexity by setting para_mode=False).
So, you will want to adjust the num_cores parameter below to the number of cores in your machine.
However, the code cannot handle a situations where you are converting more movies than there are cores.
So, if you are converting fewer movies than you have cores, then allocate the number of cores to the total number of movies.

In order for the parallel processing to work, open a terminal and activate the environment for this project (e.g., 'conda active wake' for the 'wake' environment), and run the following code, where the final number is the number of cores to be run:

> ipcluster start -n 8

You should get a message that "Engines appear to have started successfully", if things are working.

Next, execute the batch_command function below.

In [11]:
# Whether to use parallel processing 
# (set to False when running this on a small number of movies)
para_mode = False

def batch_command(cmds):
    """ Runs a series of command-line instructions (in cmds dataframe) in parallel """

    import ipyparallel as ipp
    import subprocess

    # Set up clients 
    client = ipp.Client()
    type(client), client.ids

    # Direct view allows shared data (balanced_view is the alternative)
    direct_view = client[:]

    # Function to execute the code
    def run_command(idx):
        import os
        output = os.system(cmds_run.command[idx])
        # output = subprocess.run(cmds_run.command[idx], capture_output=True)
        # result = output.stdout.decode("utf-8")
        return output
        # return idx

    direct_view["cmds_run"] = cmds

    res = []
    for n in range(len(direct_view)):
        res.append(client[n].apply(run_command, n))
    
    return res

## Step 1 of 3
The compressed videos are created in three steps. 
First, uncompressed videos are generated with a mask and saved in the 'tmp' folder (within the video directory). 
The cell below accomplishes this step, but note that the work is accomplished by sending the job to the terminal, which makes it look like here like the job is complete. 
After you have started the job, you can track its progress below.

In [12]:
# Extract experiment catalog info
cat = af.get_cat_info(path['cat'], include_mode='make_video')
if len(cat) == 0:
    raise ValueError('No videos found in catalog file. Analyze make be set to zero for all')

# Make the masked videos (stored in 'tmp' directory)
cmds = af.convert_masked_videos(cat, in_path=path['vidin'], out_path=path['tmp'], 
            maskpath=path['mask'], vmode=False, imquality=1, para_mode=para_mode, 
            echo=False, out_name='date_sch_trial')

if para_mode:
    # Run FFMPEG commands in parallel
    results = batch_command(cmds)

### For batches
For parallel mode, you can check on the status of this job with the print command in the cell below.

In [13]:
if para_mode:
    print(results)

In response to executing the above cell, you will see the following result for each video that is being worked on:

> <AsyncResult(run_command): pending>

However, if there is a problem, then you will see something like this:

> <AsyncResult(run_command): failed>

When the job finishes correctly, then it will look like this:

> <AsyncResult(run_command): finished>

You can keep re-running the cell below each time you want to check on its status. If you have a lot of movies, then this can take a long time. 
Do not move to the next step until all videos are finished.

## Step 2 of 3
In the second step, the masked videos are compressed and cropped with parallel processing.
This is achieved in a similar way, with the following cell.

In [14]:
# Thickness (in pixels) of border around ROI
border_pix = 10

# Formulate command(s) for conversion
cmds = af.convert_videos(cat, in_path=path['tmp'], out_path=path['vidout'], 
            in_name='date_sch_trial', out_name='date_sch_trial', vmode=False, imquality=vid_quality, 
            suffix_in='mp4', para_mode=para_mode, echo=False, border_pix=border_pix)

if para_mode:
    # Run FFMPEG commands in parallel
    results = batch_command(cmds)

### For batches
Again, you can check on the job status (parallel mode only):

In [15]:
if para_mode:
    print(results)

## Step 3 of 3 
Once the job is finished, then you can survey the directories to make sure that all the videos have been compressed and the tmp files will be deleted:

In [16]:
# Step through each experiment
for c_row in cat.index:

    # Define trial filename
    # trialnum = str(int(cat.trial_num[c_row]))
    # trialnum = '00' + trialnum[-3:]
    schnum = format(int(cat.sch_num[c_row]),'03')
    trialnum = format(int(cat.trial_num[c_row]),'03')
    datetrial_name = cat.date[c_row] + '_sch' + schnum + '_tr' + trialnum

    # Temp video path
    vid_tmp_path = path['tmp'] + os.sep + datetrial_name + '.mp4'

    # Output video path
    vid_out_path = path['vidout'] + os.sep + datetrial_name + '.mp4'

    # Check whether output file was not made
    if not os.path.isfile(vid_out_path):

        print('   Output movie NOT created successfully: ' + vid_out_path)

        if os.path.isfile(vid_tmp_path):
            print('   Also, temp. movie NOT created successfully: ' + vid_tmp_path)
        else:
            print('   But, temp. movie created successfully: ' + vid_tmp_path)

    # If it was . . .
    else:
        print('   Output movie created successfully: ' + vid_out_path)

        # Delete temp file
        if os.path.isfile(vid_tmp_path):
            os.remove(vid_tmp_path)

   Output movie created successfully: /vortex/schooling/TRex/scaleRN/video/compressed/2023-04-22_sch003_tr001.mp4


# TGrabs and TRex

## Parameters

TGrabs parameters are described in the [TRex documentation](https://trex.run/docs/parameters_tgrabs.html)

In [42]:
# Parameter list for TGrabs 
param_list_tgrabs = [
    ['threshold','20'],
    ['averaging_method','mode'],
    ['average_samples','150'],
    ['blob_size_range','[0.03,1.5]'],
    ['meta_conditions','scaling_exp'],
    ['meta_species','rummy_nose_tetra'],
    ['meta_misc','school_ABC']
    ]

# print(param_list_tgrabs)

# Specify list of parameter values for TRex, all listed as strings
param_list_trex = [
    ['track_threshold','20'],
    ['blob_size_ranges','[0.03,1.5]'],
    ['track_max_speed','70'],
    ['output_format','npz'],
    ['output_invalid_value','nan'],
    # ['gui_zoom_limit','[100,100]'],
    ['gui_show_posture','false'],
    ['gui_show_paths','false'],
    ['gui_show_outline', 'true'], 
    ['gui_show_midline', 'true'], 
    ['gui_show_blobs', 'true'],
    ['gui_show_number_individuals', 'true']
    ]


# TGrabs generation of movies for TRex

## Execute TGrabs
Cell below generates dv videos, which will be used by TRex, from compressed videos.
This will be completed for each row of cat.

In [40]:
# Define input parameter list as dataframe
param_input = pd.DataFrame(param_list_tgrabs, columns=['param_name', 'param_val'])
# param_input = pd.DataFrame([])

# Add the TRex parameter listing to the TGrabs parameters
# (might improve the preliminary tracking)
param_input.append(param_list_trex)

# Extract experiment catalog info
cat = af.get_cat_info(path['cat'], include_mode='analyze')

# Loop thru each video listed in cat
for c_row in cat.index:

    # Define trial filename
    trialnum = str(int(cat.trial_num[c_row]))
    trialnum = '00' + trialnum[-3:]

    schnum = str(int(cat.sch_num[c_row]))
    schnum = '00' + schnum[-3:]

    datetrial_name = cat.date[c_row] + '_sch' + schnum + '_tr' + trialnum

    # Define and check input path
    path_in = path['vidout'] + os.sep + datetrial_name + '.mp4'
    if not os.path.isfile(path_in):
        raise OSError('Video file does not exist: ' + path_in)

    # Output path
    path_out = path['vidpv'] + os.sep + datetrial_name + '.pv'

    # Path to save settings table
    path_settings = path['settings'] + os.sep + datetrial_name + '_tgrabs_settings.csv'

    # Start formulating the TGrabs command
    command = f'tgrabs -i {path_in} -o {path_out} '

    # Add additional command
    # command += '-averaging_method mode '

     # Get max number of fish from spreadsheet
    command += f'-track_max_individuals {str(int(cat.fish_num[c_row]))} '

     # Get real width of processed frame (in cm) from spreadsheet
    command += f'-meta_real_width {str(int(cat.fr_width_cm[c_row]))} '

    # Loop thru each parameter value included in cat
    for idx in param_input.index:
        command += '-' + str(param_input.param_name[idx]) + ' ' + str(param_input.param_val[idx]) + ' '

    # Write settings to csv
    param_input.to_csv(path_settings, index=False)
    
    # Execute at the command line
    #os.system(command)
    print(command)

tgrabs -i /vortex/schooling/TRex/scaleRN/video/compressed/2023-04-22_sch001_tr006.mp4 -o /vortex/schooling/TRex/scaleRN/video/pv/2023-04-22_sch001_tr006.pv -track_max_individuals 20 -meta_real_width 107 -threshold 20 -averaging_method mode -average_samples 150 -blob_size_range [0.03,1.5] -meta_conditions scaling_exp -meta_species rummy_nose_tetra -meta_misc school_ABC 


# Running TRex

## Batch execution of experiment videos with TRex

Be sure that experiment_log.csv includes the name of the settings file, in the settings directory, that TRex can use to 

In [43]:
# Create the pandas DataFrame that holds the parameter values
params_trex = pd.DataFrame(param_list_trex, 
                    columns=['param_name', 'param_val'])

# Extract experiment catalog info
cat = af.get_cat_info(path['cat'], include_mode='analyze')

# Loop thru each video listed in cat
for c_row in cat.index:

    # Define trial filename
    trialnum = str(int(cat.trial_num[c_row]))
    trialnum = '00' + trialnum[-3:]

    schnum = str(int(cat.sch_num[c_row]))
    schnum = '00' + schnum[-3:]

    datetrial_name = cat.date[c_row] + '_sch' + schnum + '_tr' + trialnum

    # Define and check input path
    path_in = path['vidpv'] + os.sep + datetrial_name + '.pv'

    # Check for input file
    if not os.path.isfile(path_in):
        raise OSError('Video file does not exist: ' + path_in)
    
    # Where data will be saved
    data_path = path['data_raw'] 

    # Overwrite number of fish
    # params_trex.track_max_individuals = str(int(cat.fish_num[c_row]))

    # Settings path
    path_settings = path['settings'] + os.sep + datetrial_name + '.settings'
    #path_settings = datetrial_name + '.settings'

    # Start formulating the TGrabs command
    #command = f'trex -i {path_in} -output_dir {data_path} -settings_file {path_settings} '
    command = f'trex -i {path_in} -output_dir {data_path} '

    # Add path for settings
    # command += f'-settings_file {path_settings} '

    # Get max number of fish from spreadsheet
    command += f'-track_max_individuals {str(int(cat.fish_num[c_row]))} '

    # Loop thru each parameter value included in cat
    for idx in params_trex.index:
        command += '-' + str(params_trex.param_name[idx]) + ' ' + str(params_trex.param_val[idx]) + ' '

    # Execute at the command line
    result = os.system(command)
    #print(command)

[18:08:55] ------------------------
[18:08:55] LOADING DEFAULT SETTINGS
[18:08:55] ------------------------
[18:08:55] Found ffmpeg in "/home/anpetey/anaconda3/envs/TREX/bin/ffmpeg"
[18:08:55] Property<path>('ffmpeg_path') = "/home/anpetey/anaconda3/envs/TREX/bin/ffmpeg"
[18:08:55] Property<string>('build') = "4ce4be1cd1679bcca4da5af8780dd86c1d31050b"
[18:08:55] Property<string>('cmd_line') = " trex -i /vortex/schooling/TRex/scaleRN/video/pv/2023-04-22_sch001_tr006.pv -output_dir /vortex/schooling/TRex/scaleRN/data/raw -track_max_individuals 20 -track_threshold 20 -blob_size_ranges [0.03,1.5] -track_max_speed 70 -output_format npz -output_invalid_value nan -gui_show_posture false -gui_show_paths false -gui_show_outline true -gui_show_midline true -gui_show_blobs true -gui_show_number_individuals true"
[18:08:55] -------------------
[18:08:55] LOADING COMMANDLINE
[18:08:55] -------------------
[18:08:55] Property<path>('wd') = "/home/anpetey/anaconda3/envs/TREX/bin"
[18:08:55] ---------

2023-04-27 11:24:40.354292: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-04-27 11:24:42.387788: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-27 11:24:42.389812: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-04-27 11:24:42.389961: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-04-27 11:24:42.892326: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:19:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.77GHz core

setting version 3.7.13 (default, Mar 29 2022, 02:21:02) 
[GCC 7.5.0] True Quadro RTX 8000
39] 	Finding more (65) validation images to reach 85 samples from 321 available images.
[18:24:39] 	Selected 65 new images (85 / 85)
[18:24:39] 	Finding more (63) validation images to reach 85 samples from 318 available images.
[18:24:39] 	Selected 52 new images (74 / 85)
[18:24:39] 	Finding more (54) validation images to reach 84 samples from 307 available images.
[18:24:39] 	Selected 56 new images (86 / 84)
[18:24:39] 	Finding more (85) validation images to reach 85 samples from 340 available images.
[18:24:39] 	Selected 78 new images (78 / 85)
[18:24:39] 	Finding more (69) validation images to reach 85 samples from 325 available images.
[18:24:39] 	Selected 77 new images (93 / 85)
[18:24:39] 	Finding more (84) validation images to reach 84 samples from 339 available images.
[18:24:39] 	Selected 79 new images (79 / 84)
[18:24:39] 	Finding more (47) validation images to reach 84 samples from 300 

2023-04-27 11:24:46.894384: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-04-27 11:24:46.918468: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2299965000 Hz
2023-04-27 11:24:47.129413: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-04-27 11:24:53.379244: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


  4/256 [..............................] - ETA: 14s - loss: 0.8363 - accuracy: 0.0107     

...

256/256 [==============================] - 25s 63ms/step - loss: 0.5650 - accuracy: 0.1992 - val_loss: 0.7416 - val_accuracy: 0.0852
Epoch 2/150
256/256 [==============================] - 15s 59ms/step - loss: 0.2829 - accuracy: 0.4886 - val_loss: 0.5700 - val_accuracy: 0.1554
Epoch 3/150
256/256 [==============================] - 15s 58ms/step - loss: 0.1844 - accuracy: 0.6302 - val_loss: 0.1158 - val_accuracy: 0.7679
[18:24:45] [py] Total params: 269,420
[18:24:45] [py] Trainable params: 268,860
[18:24:45] [py] Non-trainable params: 560
[18:24:45] [py] _________________________________________________________________
[18:24:45] Pushing 24577 images (629.17MB) to python...
[18:24:45] [py] setting move_range as 0.025
[18:24:45] [py] {'epochs': 150, 'batch_size': 64, 'move_range': 0.025, 'output_path': '/vortex/schooling/TRex/scaleRN/data/raw/2023-04-22_sch001_tr006_weights', 'image_width': 80.0, 'image_height': 80.0, 'classes': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,

Traceback (most recent call last):
  File "/home/anpetey/anaconda3/envs/TREX/usr/share/trex/learn_static.py", line 565, in start_learning
    raise Exception("aborting with error")
Exception: aborting with error


[EXCEPT 18:36:42 GPURecognition.cpp:870] Python runtime exception while running learn_static::start_learning(): Exception: aborting with error

At:
  /home/anpetey/anaconda3/envs/TREX/usr/share/trex/learn_static.py(565): start_learning

[18:36:42] Runtime error: Python runtime exception while running learn_static::start_learning(): Exception: aborting with error

At:
  /home/anpetey/anaconda3/envs/TREX/usr/share/trex/learn_static.py(565): start_learning

[18:36:42] Training the network failed (-1).
[18:36:42] [STEP] <key>0</key> (<nr>-100</nr>%, 0 added): <b><str>Failed</str></b>. Training routine returned error code. 

[EXCEPT 18:36:42 Accumulation.cpp:882] [Restart] Initial training failed. Cannot continue to accumulate.
[18:36:42] Writing backup of settings...
[18:36:42] Saved "/vortex/schooling/TRex/scaleRN/data/raw/backup/2023-04-22_sch001_tr006.settings".
[18:36:42] Writing backup of settings...
[18:36:42] Saved "/vortex/schooling/TRex/scaleRN/data/raw/backup/2023-04-22_sch001_tr

2023-04-27 11:39:11.766307: W tensorflow/core/kernels/data/generator_dataset_op.cc:107] Error occurred when finalizing GeneratorDataset iterator: Failed precondition: Python interpreter state is not initialized. The process may be terminated.
	 [[{{node PyFunc}}]]


[18:37:20] Property<bool>('analysis_paused') = true
[18:37:29] Property<array<uint>>('heatmap_ids') = [14]
[18:37:29] Property<array<uint>>('heatmap_ids') = [14,18]
[18:37:30] Property<array<uint>>('heatmap_ids') = []
[18:37:30] Property<array<uint>>('heatmap_ids') = [19]
[18:37:31] Property<array<uint>>('heatmap_ids') = []
[18:37:31] Property<array<uint>>('heatmap_ids') = [19]
[18:37:31] Property<array<uint>>('heatmap_ids') = []
[18:37:31] Property<array<uint>>('heatmap_ids') = [14]
[18:37:31] Property<array<uint>>('heatmap_ids') = [14,18]
[18:37:38] Property<array<uint>>('heatmap_ids') = []
[18:37:38] Property<array<uint>>('heatmap_ids') = [14]
[18:37:38] Property<array<uint>>('heatmap_ids') = [14,18]
[18:37:39] Property<array<uint>>('heatmap_ids') = []
[18:37:39] Property<array<uint>>('heatmap_ids') = [14]
[18:37:39] Property<array<uint>>('heatmap_ids') = [14,18]
[18:37:40] Property<array<uint>>('heatmap_ids') = []
[18:37:40] Property<array<uint>>('heatmap_ids') = [14]
[18:37:40] Pr

## Export data in mat format

In [ ]:
from scipy.io import savemat
import glob

# Extract experiment catalog info 
cat = af.get_cat_info(path['cat'], include_mode='analyze')


# Convert all npz files for an experiment to mat files.
da.raw_to_mat(cat)